In [2]:
import json
import time
import os
import glob
import requests

In [3]:
os.chdir("../../")

In [4]:
from src.utils.test_images import image_links
IMAGE_LINKS = image_links

## Step 2:  Load Pre-trained Model from TensorFlow Hub

In [6]:
import tensorflow as tf
import tensorflow_hub as hub

In [7]:
IMAGE_SHAPE = (224, 224)

In [8]:
def load_model():
    mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
    classifier_model = mobilenet_v2

    classifier = tf.keras.Sequential([
        hub.KerasLayer(classifier_model, input_shape=IMAGE_SHAPE + (3,))
    ])
    print("Model loaded")
    return classifier

## Step 3: Save the loaded Model

In [9]:
def save_model():
    model = load_model()
    ts = int(time.time())
  
    base_file_path = os.getcwd() + "/img-classifier-tfx/src/models/"
    print(base_file_path)

    file_path = base_file_path + str(ts)
    model.save(filepath=file_path, save_format='tf')
    print(f"Model Saved under {file_path}")


In [10]:
from src.utils.test_images import image_links
from src.utils.utilities import is_dir_exists, is_directory_empty

In [14]:
MODEL_LOCATION = "/home/murali/Documents/img-classifier-tfx-main/src/models"
MODEL_LOCATION

'/home/murali/Documents/img-classifier-tfx-main/src/models'

In [15]:
if not is_dir_exists(MODEL_LOCATION):
    # Create a new directory because it does not exist
    os.makedirs(MODEL_LOCATION)
if is_directory_empty(MODEL_LOCATION):
    # Load and save the model when pre-loaded model does not exist
    save_model()
else:
    print("Model exists")


Model exists


## Step 4: Serve TF Model using TF Serving

In [16]:
from src.app.tf_serve import preprocess_image, decode_predictions




In [17]:
from src.app.tf_serve import get_model_name


In [19]:
model_name = get_model_name(MODEL_LOCATION)
# URL at which the model is being served
URL = f'http://localhost:8501/v1/models/{model_name}:predict'
URL

'http://localhost:8501/v1/models/1702385256:predict'

In [79]:

def serve_rest(img_url):
    img = preprocess_image(img_url)
    print("Img preprocessing done!")
    
    data = json.dumps({"signature_name": "serving_default", "instances": img.tolist()})
    headers = {"content-type": "application/json"}
    
    json_response = requests.post(URL, data=data, headers=headers)
    
    predictions = json.loads(json_response.text)['predictions']
    
    results = decode_predictions(predictions)
    print("predictions decoded")
    
    return results


## Step 5: Run TF Serving


## Predict Results

In [80]:

for IMAGE_LINK in IMAGE_LINKS:
        image_url = IMAGE_LINK["url"]
        print(f"Image Link: {image_url}")
        res = serve_rest(image_url)
        print(res)
        print()
    
       


Image Link: https://github.com/pytorch/hub/raw/master/images/dog.jpg
Img preprocessing done!
10484/10484 [==============================] - 0s 8us/step
predictions decoded
{'predicted_label': 'Samoyed', 'probability': 0.6407572031021118}

Image Link: https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/1200px-Cat03.jpg
Img preprocessing done!
predictions decoded
{'predicted_label': 'tiger cat', 'probability': 0.6095588803291321}

Image Link: https://media.istockphoto.com/photos/hen-picture-id831791190?k=20&m=831791190&s=170667a&w=0&h=pT2buEaMcxE5Wm4hQz-pXl0Zv-0DdpSMwpDzVcpcKyo=
Img preprocessing done!
predictions decoded
{'predicted_label': 'hen', 'probability': 0.9590941071510315}

